In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/MyDrive/Health Classification'

import numpy as np
import pandas as pd

Mounted at /content/drive
/content/drive/MyDrive/Health Classification


create silver dataset

In [ ]:
ep=pd.read_csv('/content/drive/MyDrive/Health Classification/Health classification EP/classification_results.csv')
com=pd.read_csv('/content/drive/MyDrive/Health Classification/Health classification Com/classification_results.csv')
council=pd.read_csv('/content/drive/MyDrive/Health Classification/Health classification Council/classification_results.csv')

ep=ep[['current',
       'is_climate_LLM', 'llm_explanation']].rename(columns={'current': 'sentence'})
com=com[['sentence_text_en',
       'is_climate_LLM', 'llm_explanation']].rename(columns={'sentence_text_en': 'sentence'})
council=council[['sentence',
       'is_climate_LLM', 'llm_explanation']].rename(columns={'sentence': 'sentence'})

merged_df = pd.concat([ep, com, council], ignore_index=True)
display(merged_df.head())
print(merged_df.shape)

,sentence,is_climate_LLM,llm_explanation
0,Setting up HERA as soon as possible stems from...,1.0,1\n\nThis sentence is about health in the EU p...
1,"Mr President, I want first to address the Comm...",0.0,0\n\nThis sentence is not about health in the ...
2,It will be an important contribution to preven...,1.0,1\n\nThis sentence is about health in the EU p...
3,We will launch a call for proposals this month...,1.0,**1**\n\nThis sentence is about health in the ...
4,With regard to the final remark concerning the...,1.0,1\n\nThis sentence is about health in the EU p...


(30000, 3)


In [ ]:
# Convert '-' to NaN in 'bartu_annotation'
merged_df['is_climate_LLM'] = merged_df['is_climate_LLM'].replace('-', np.nan)

# Drop rows where either 'bartu_annotation' or 'is_climate_LLM' is NaN
merged_df.dropna(subset=[ 'is_climate_LLM'], inplace=True)

# Convert columns to integer
merged_df['is_climate_LLM'] = merged_df['is_climate_LLM'].astype(int)


# Display the data types of the columns to confirm
print(merged_df.dtypes)
display(merged_df.head())

merged_df.to_csv('silver_all.csv', index=False)

sentence           object
is_climate_LLM      int64
llm_explanation    object
dtype: object


,sentence,is_climate_LLM,llm_explanation
0,Setting up HERA as soon as possible stems from...,1,1\n\nThis sentence is about health in the EU p...
1,"Mr President, I want first to address the Comm...",0,0\n\nThis sentence is not about health in the ...
2,It will be an important contribution to preven...,1,1\n\nThis sentence is about health in the EU p...
3,We will launch a call for proposals this month...,1,**1**\n\nThis sentence is about health in the ...
4,With regard to the final remark concerning the...,1,1\n\nThis sentence is about health in the EU p...


In [ ]:
# Install Hugging Face libraries
!pip install -q transformers datasets evaluate

# --------------------------
# 1. Imports
# --------------------------
from datasets import load_dataset, DatasetDict, concatenate_datasets
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
import evaluate
import numpy as np
import os


In [ ]:

# --------------------------
# 2. Load Datasets
# --------------------------
print("Loading datasets...")

# Load Gold data
gold_all = load_dataset("csv", data_files="gold_all.csv")["train"]

# Load Silver data (with 'is_climate_llm' label column)
silver_all = load_dataset("csv", data_files="silver_all.csv")["train"]  # Replace with your silver data file

print(f"Original gold dataset size: {len(gold_all)}")
print(f"Original silver dataset size: {len(silver_all)}")


Loading datasets...


Generating train split: 0 examples [00:00, ? examples/s]

Original gold dataset size: 2101
Original silver dataset size: 29897


In [ ]:

# --------------------------
# 3. Process Gold Data (Binary Classification)
# --------------------------
# Count occurrences of each class in gold data
gold_class_counts = gold_all.to_pandas()['bartu_annotation'].value_counts()
print("\nGold data class distribution:")
print(gold_class_counts)

# For binary classification (0, 1), no need to filter single-member classes
# Convert the 'bartu_annotation' column to ClassLabel on the gold dataset
gold_all_filtered = gold_all.class_encode_column("bartu_annotation")

# Binary classification: 2 labels (0, 1)
num_labels = 2
label_names = [0, 1]
print(f"Number of labels: {num_labels}")
print(f"Label names: {label_names}")

# Perform the stratified train/validation/test split on the filtered gold dataset
train_valid_test = gold_all_filtered.train_test_split(test_size=0.3, stratify_by_column="bartu_annotation")
temp = train_valid_test['test'].train_test_split(test_size=0.5, stratify_by_column="bartu_annotation")

gold_data = DatasetDict({
    'train': train_valid_test['train'].rename_column("bartu_annotation", "labels"),
    'validation': temp['train'].rename_column("bartu_annotation", "labels"),
    'test': temp['test'].rename_column("bartu_annotation", "labels")
})

print(f"Gold train set size: {len(gold_data['train'])}")
print(f"Gold validation set size: {len(gold_data['validation'])}")
print(f"Gold test set size: {len(gold_data['test'])}")



Gold data class distribution:
bartu_annotation
0    1232
1     869
Name: count, dtype: int64
Number of labels: 2
Label names: [0, 1]
Gold train set size: 1470
Gold validation set size: 315
Gold test set size: 316


In [ ]:

# --------------------------
# 4. Process Silver Data (Binary Classification)
# --------------------------
print("\nProcessing silver data...")

# Silver data has binary labels (0, 1) in 'is_climate_llm' column
# Apply class encoding to silver data
silver_filtered = silver_all.class_encode_column("is_climate_LLM")

# Rename column for consistency
silver_data = silver_filtered.rename_column("is_climate_LLM", "labels")

print(f"Silver dataset size: {len(silver_data)}")

# Check silver data class distribution
silver_class_counts = silver_data.to_pandas()['labels'].value_counts()
print("Silver data class distribution:")
print(silver_class_counts)



Processing silver data...


Stringifying the column:   0%|          | 0/29897 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/29897 [00:00<?, ? examples/s]

Silver dataset size: 29897
Silver data class distribution:
labels
0    21164
1     8733
Name: count, dtype: int64


In [ ]:

# --------------------------
# 5. Tokenization
# --------------------------
model_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch["sentence"], padding="max_length", truncation=True, max_length=256)

# Tokenize all datasets
print("Tokenizing datasets...")
gold_data = gold_data.map(tokenize, batched=True)
gold_data = gold_data.remove_columns(["sentence"])

silver_data = silver_data.map(tokenize, batched=True)
silver_data = silver_data.remove_columns(["sentence"])


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Tokenizing datasets...


Map:   0%|          | 0/1470 [00:00<?, ? examples/s]

Map:   0%|          | 0/315 [00:00<?, ? examples/s]

Map:   0%|          | 0/316 [00:00<?, ? examples/s]

Map:   0%|          | 0/29897 [00:00<?, ? examples/s]

In [ ]:

# --------------------------
# 6. STAGE 1: Pre-training on Silver Data
# --------------------------
print("\n" + "="*50)
print("STAGE 1: Pre-training on Silver Data")
print("="*50)

# Load model for pre-training
model_silver = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {"f1": metric.compute(predictions=preds, references=labels, average='weighted')["f1"]}

# Split silver data for pre-training (use 90% for training, 10% for validation)
silver_split = silver_data.train_test_split(test_size=0.1, stratify_by_column="labels")

# Pre-training arguments
pretraining_args = TrainingArguments(
    output_dir="./results_silver_pretraining",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,  # Fewer epochs for pre-training
    weight_decay=0.01,
    load_best_model_at_end=True,
    logging_dir="./logs_silver_pretraining",
    save_total_limit=2,
    metric_for_best_model="f1",
    greater_is_better=True,
    logging_steps=100
)

# Pre-training trainer
pretrainer = Trainer(
    model=model_silver,
    args=pretraining_args,
    train_dataset=silver_split['train'],
    eval_dataset=silver_split['test'],
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

# Train on silver data
print("Starting pre-training on silver data...")
pretrainer.train()

# Save the pre-trained model
pretrained_model_dir = "./pretrained_on_silver"
pretrainer.save_model(pretrained_model_dir)
print(f"Pre-trained model saved to {pretrained_model_dir}")



STAGE 1: Pre-training on Silver Data


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Starting pre-training on silver data...


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: bartuturan0 (bartuturan0-technical-university-of-munich) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,F1
1,0.155700,0.133474,0.953563
2,0.133300,0.139673,0.955892


Pre-trained model saved to ./pretrained_on_silver


In [ ]:

# --------------------------
# 7. STAGE 2: Fine-tuning on Gold Data
# --------------------------
print("\n" + "="*50)
print("STAGE 2: Fine-tuning on Gold Data")
print("="*50)

# Load the pre-trained model for fine-tuning
model_gold = AutoModelForSequenceClassification.from_pretrained(pretrained_model_dir)

# Fine-tuning arguments (typically use lower learning rate)
finetuning_args = TrainingArguments(
    output_dir="./results_gold_finetuning",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-5,  # Lower learning rate for fine-tuning
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    logging_dir="./logs_gold_finetuning",
    save_total_limit=2,
    metric_for_best_model="f1",
    greater_is_better=True,
    logging_steps=50
)

# Fine-tuning trainer
finetuner = Trainer(
    model=model_gold,
    args=finetuning_args,
    train_dataset=gold_data['train'],
    eval_dataset=gold_data['validation'],
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

# Fine-tune on gold data
print("Starting fine-tuning on gold data...")
finetuner.train()



STAGE 2: Fine-tuning on Gold Data
Starting fine-tuning on gold data...


Epoch,Training Loss,Validation Loss,F1
1,0.225700,0.162591,0.949505
2,0.126100,0.176757,0.949505
3,0.131600,0.174996,0.949476


TrainOutput(global_step=276, training_loss=0.14621532222499017, metrics={'train_runtime': 252.1022, 'train_samples_per_second': 17.493, 'train_steps_per_second': 1.095, 'total_flos': 580159877068800.0, 'train_loss': 0.14621532222499017, 'epoch': 3.0})

In [ ]:

# --------------------------
# 8. Final Evaluation
# --------------------------
print("\n" + "="*50)
print("FINAL EVALUATION")
print("="*50)

# Evaluate on gold test set
results = finetuner.evaluate(gold_data['test'])
print("Final Results on Gold Test Set:", results)

# Save the final fine-tuned model
final_model_dir = "./final_model_silver_pretrained_gold_finetuned"
finetuner.save_model(final_model_dir)
print(f"Final model saved to {final_model_dir}")



FINAL EVALUATION


Final Results on Gold Test Set: {'eval_loss': 0.1924026757478714, 'eval_f1': 0.9400245747956375, 'eval_runtime': 4.1861, 'eval_samples_per_second': 75.489, 'eval_steps_per_second': 4.778, 'epoch': 3.0}
Final model saved to ./final_model_silver_pretrained_gold_finetuned
